# 1. Import libraries

In [3]:

from llama_parse import LlamaParse
from llama_index.core import Settings
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from IPython.display import Markdown, display
from llama_index.postprocessor.cohere_rerank import CohereRerank
from pinecone import Pinecone, ServerlessSpec


# llama-parse is async-first, this is needed in order to run async code in notebooks
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

True

# 2. Pinecone Setup

In [4]:
os.getenv('LLAMA_CLOUD_API_KEY')

'llx-Js0RSa1fd9oSzDD4exEcAgo10t8uvAAwoDGwQXdu5lPLkuBg'